<a href="https://colab.research.google.com/github/abhiyantaabhishek/COVID_19_Detection/blob/master/COVID_19_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import numpy as np

In [2]:
!git clone https://github.com/abhiyantaabhishek/COVID_19_Detection

Cloning into 'COVID_19_Detection'...
remote: Enumerating objects: 2, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 750 (delta 0), reused 2 (delta 0), pack-reused 748
Receiving objects: 100% (750/750), 85.65 MiB | 22.28 MiB/s, done.


In [3]:
ls

COVID_19_Detection/  sample_data/


In [4]:
%cd COVID_19_Detection

/content/COVID_19_Detection


In [8]:
CT_COVID = os.listdir('CT_COVID')

In [10]:
CT_NonCOVID = os.listdir('CT_NonCOVID')

In [13]:
CT_COVID_label =  [1]*len(CT_COVID)

In [16]:
CT_NonCOVID_label = [0]*len(CT_NonCOVID)

In [19]:
CT_COVID_label = np.array(CT_COVID_label)
CT_NonCOVID_label = np.array(CT_NonCOVID_label)